In [1]:
from __future__ import division
import pandas as pd
from datetime import datetime, timedelta, date
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import xgboost as xgb
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold, cross_val_score, train_test_split

import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go

#initiate plotly
pyoff.init_notebook_mode()

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
df = pd.read_excel("data/Online_Retail.xlsx", sheet_name='Online Retail')
df.head(10)

In [ ]:
#convert string date into datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

#using only data from the UK
df_uk = df.query("Country == 'United Kingdom' ").reset_index(drop=True)
df_uk.head(10)

In [ ]:
# Divide the dataset in 2
# 3 months to calculate RFM (Recency, Frequency, and Monetary)
# and 6 month for predicting

#df_3m = df_uk[(df_uk.InvoiceDate < date(2011,6,1)) & (df_uk.InvoiceDate >= date (2011,3,1))].reset_index(drop=True)
#df_6m = df_uk[(df_uk.InvoiceDate >= date(2011,6,1)) & (df_uk.InvoiceDate <  date (2011,12,1))].reset_index(drop=True)

df_3m = df_uk[(df_uk.InvoiceDate < pd.to_datetime('2011-06-01')) & (df_uk.InvoiceDate >= pd.to_datetime('2011-03-01'))].reset_index(drop=True)
df_6m = df_uk[(df_uk.InvoiceDate >= pd.to_datetime('2011-06-01')) & (df_uk.InvoiceDate < pd.to_datetime('2011-12-01'))].reset_index(drop=True)



#create df_user for assigning clustering
#Grouping together similar data points into clusters or groups. The goal is to identify patterns & r/ships within a dataset.

df_user = pd.DataFrame(df_3m['CustomerID'].unique())
df_user.columns = ['CustomerID']

In [ ]:
#order cluster
def order_cluster(cluster_field_name, target_field_name, df, ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name, ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df, df_new[[cluster_field_name, 'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name], axis = 1)
    df_final = df_final.rename(columns={"index": cluster_field_name})
    return  df_final

In [ ]:
# Calculate the recency score -- ref: 3 months to calculate RFM (Recency, Frequency, and Monetary)

df_max_purchase = df_3m.groupby('CustomerID').InvoiceDate.max().reset_index()
df_max_purchase.columns = ['CustomerID', 'MaxPurchaseDate']
df_max_purchase['Recency'] = (df_max_purchase['MaxPurchaseDate'].max() - df_max_purchase['MaxPurchaseDate']).dt.days
df_user = pd.merge(df_user, df_max_purchase[['CustomerID', 'Recency']], on= 'CustomerID')
df_user.head()

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(df_user[['Recency']])
df_user['RecencyCluster'] = kmeans.predict(df_user[['Recency']])

df_user = order_cluster('RecencyCluster', 'Recency', df_user, False)

In [ ]:
# Calculate the Frequency score -- ref: 3 months to calculate RFM (Recency, Frequency, and Monetary)
df_frequency = df_3m.groupby('CustomerID').InvoiceDate.count().reset_index()
df_frequency.columns = ['CustomerID', 'Frequency']
df_user = pd.merge(df_user, df_frequency, on='CustomerID')